In [53]:
from functools import reduce
import requests

In [54]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [55]:
from tqdm.notebook import tqdm

## Open County data

In [56]:
file_path = "data/cb_2024_us_county_500k/cb_2024_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)

## Get Ethnic Data

In [57]:
r = requests.get("https://api.census.gov/data/2023/acs/acs5/groups/B05006.json")
columns_obj = r.json()

### Get columns to query and rename for later

In [58]:
columns = []
rename_vars = {}
variables = columns_obj["variables"]
for name, variable in list(variables.items()):
    v_split = variable["label"].split("!!")
    if len(v_split) < 4:
        continue

    if v_split[0] == "Estimate":
        label = v_split[-1]
        rename_vars[name] = label

    if name.endswith("E") or name.endswith("M"):
        columns.append(name)

In [59]:
dfs = []
for i in tqdm(range(0, len(columns), 49), desc="Requesting data in batches"):
    columns_with_geoid = columns[i : i + 49]
    columns_with_geoid.append("GEO_ID")
    columns_formatted = ",".join(columns_with_geoid)
    url = f"https://api.census.gov/data/2023/acs/acs5?get={columns_formatted}&for=county:*"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    dfs.append(df)

Requesting data in batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [60]:
dfs_clean = [
    df.drop(columns=["state", "county"], errors="ignore") if i > 0 else df
    for i, df in enumerate(dfs)
]

In [61]:
df = reduce(
    lambda left, right: pd.merge(left, right, on="GEO_ID", how="inner"), dfs_clean
)

In [62]:
df = df.replace(np.nan, 0)

In [63]:
estimate_cols = [col for col in df.columns if col.endswith("E")]

In [64]:
formtted_df = df[["GEO_ID", *estimate_cols]]
formtted_df[estimate_cols] = formtted_df[estimate_cols].astype(int)

/tmp/ipykernel_30479/4148564808.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formtted_df[estimate_cols] = formtted_df[estimate_cols].astype(int)


In [65]:
filtered_estimate_cols = {
    k: v
    for k, v in rename_vars.items()
    if k in estimate_cols
    and (":" not in v or v == "United Kingdom (inc. Crown Dependencies):")
}

In [66]:
formtted_df["most_common_european_origin_raw"] = formtted_df[
    list(filtered_estimate_cols.keys())
].idxmax(axis=1)

/tmp/ipykernel_30479/42615791.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  formtted_df["most_common_european_origin_raw"] = formtted_df[
/tmp/ipykernel_30479/42615791.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formtted_df["most_common_european_origin_raw"] = formtted_df[


In [67]:
def check_margin_error(row) -> str:
    geo_id = row["GEO_ID"]
    ethnicity_col = row["most_common_european_origin_raw"]
    val = row[ethnicity_col]

    if not val:
        return None

    moe_col = ethnicity_col.replace("E", "M")
    moe_val = int(df[df["GEO_ID"] == geo_id][moe_col])

    rmoe_val = abs(moe_val / val)
    if rmoe_val < 0.50:
        return variables[ethnicity_col]["label"].split("!!")[-1]
    else:
        return None

In [68]:
formtted_df["most_common_european_origin"] = formtted_df.apply(
    lambda row: check_margin_error(row), axis=1
)

/tmp/ipykernel_30479/3273856069.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  moe_val = int(df[df["GEO_ID"] == geo_id][moe_col])
/tmp/ipykernel_30479/3496989676.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  formtted_df["most_common_european_origin"] = formtted_df.apply(
/tmp/ipykernel_30479/3496989676.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formtted_df["most_common_european_origin"] = formtted_df.app

In [70]:
formtted_df.groupby("most_common_european_origin").size().reset_index(
    name="COUNT"
).sort_values("COUNT", ascending=False)

,most_common_european_origin,COUNT
32,Mexico,1164
22,India,83
7,Canada,62
37,Philippines,50
8,"China, excluding Hong Kong and Taiwan",41
14,El Salvador,28
12,Dominican Republic,23
18,Guatemala,18
21,Honduras,15
10,Cuba,13


In [ ]:
rename_vars["GEO_ID"] = "GEOIDFQ"
formtted_df = formtted_df.rename(columns=rename_vars)

## Merge Data

In [71]:
gdf = counties_gdf.merge(formtted_df, on="GEOIDFQ", how="inner")

In [77]:
gdf[gdf["STUSPS"] == "FL"]["most_common_european_origin"].unique()

array([None, 'Jamaica', 'Mexico', 'Cuba', 'Philippines', 'Colombia',
       'Canada', 'India', 'Venezuela', 'Haiti', 'Honduras', 'Guatemala',
       'Portugal'], dtype=object)

In [76]:
gdf.groupby("STUSPS")["most_common_european_origin"].nunique().reset_index(
    name="COUNT"
).sort_values("COUNT", ascending=False)

,STUSPS,COUNT
34,NY,15
38,PA,13
9,FL,12
24,MO,10
22,MI,9
46,VA,8
35,OH,8
23,MN,7
20,MD,7
15,IN,7


In [19]:
gdf = gdf.to_crs(9311)
gdf.to_file("data/Citizen_European_Origin_Region_Per_County.gpkg")